# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("/home/workspace/Project1/data/song_data")

In [5]:
song_files

['/home/workspace/Project1/data/song_data/A/B/C/TRABCRU128F423F449.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCUQ128E0783E2B.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCYE128F934CE1D.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCEC128F426456E.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCXB128F4286BD3.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCPZ128F4275C32.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCKL128F423A778.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/workspace/Project1/data/song_data/A/B/C/TRABCFL128F149BB0D.json',
 '/home/workspace/Project1/data/song_data/A/B/B/TRABBBV128F42967D7.json',
 '/home/workspace/Project1/data/song_d

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values
song_data = list(song_data[0])
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
song_table_insert

' INSERT INTO songs (song_id, title, artist_id ,                                             year, duration)                          VALUES (%s, %s, %s, %s, %s)\n                         ON CONFLICT (song_id) DO NOTHING\n'

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].fillna(0).values
artist_data = list(artist_data[0])

In [12]:
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', 0.0, 0.0]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files("/home/workspace/Project1/data/log_data")

In [15]:
filepath = log_files[0]

In [16]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df = df[df.page == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [18]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [19]:
time_data = (df.ts, t.dt.hour, t.dt.day,t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ("time_start", "hour", "day", "weekofyear", "month", "year", "weekday")
time_data = dict(zip(column_labels, time_data))

In [20]:
time_data

{'time_start': 0      1543449657796
 1      1543449690796
 2      1543449841796
 3      1543449895796
 4      1543450033796
 5      1543450256796
 6      1543450264796
 7      1543450450796
 8      1543450562796
 9      1543450691796
 10     1543450933796
 11     1543451175796
 12     1543451453796
 13     1543451707796
 14     1543452014796
 15     1543452253796
 16     1543452394796
 17     1543452588796
 18     1543453030796
 19     1543453332796
 20     1543453596796
 21     1543453763796
 22     1543453798796
 23     1543454037796
 24     1543454254796
 25     1543454533796
 26     1543454778796
 27     1543455073796
 28     1543455218796
 29     1543455510796
            ...      
 333    1543527047796
 334    1543527094796
 336    1543527265796
 337    1543527268796
 338    1543527488796
 339    1543527730796
 340    1543528009796
 341    1543528377796
 342    1543528636796
 343    1543528831796
 344    1543529196796
 345    1543529397796
 346    1543529417796
 347    1543529595

In [21]:
time_df = pd.DataFrame(data=time_data)
time_df.head()

,time_start,hour,day,weekofyear,month,year,weekday
0,1543449657796,0,29,48,11,2018,3
1,1543449690796,0,29,48,11,2018,3
2,1543449841796,0,29,48,11,2018,3
3,1543449895796,0,29,48,11,2018,3
4,1543450033796,0,29,48,11,2018,3


In [22]:
column_labels

('time_start', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday')

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df.head()


,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [27]:
df[["ts","userId", "level", "sessionId", "location"]]

,ts,userId,level,sessionId,location
0,1543449657796,73,paid,954,"Tampa-St. Petersburg-Clearwater, FL"
1,1543449690796,24,paid,984,"Lake Havasu City-Kingman, AZ"
2,1543449841796,24,paid,984,"Lake Havasu City-Kingman, AZ"
3,1543449895796,73,paid,954,"Tampa-St. Petersburg-Clearwater, FL"
4,1543450033796,24,paid,984,"Lake Havasu City-Kingman, AZ"
5,1543450256796,24,paid,984,"Lake Havasu City-Kingman, AZ"
6,1543450264796,73,paid,954,"Tampa-St. Petersburg-Clearwater, FL"
7,1543450450796,73,paid,954,"Tampa-St. Petersburg-Clearwater, FL"
8,1543450562796,24,paid,984,"Lake Havasu City-Kingman, AZ"
9,1543450691796,24,paid,984,"Lake Havasu City-Kingman, AZ"


In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song.replace("'", "''"), row.artist.replace("'", "''"), row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (songid, artistid, row.ts, row.userId, row.level,row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.